In [4]:
## Import the essential library

import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

In [5]:
# Load the enviornment
load_dotenv()

True

In [6]:
#Get the Twitter bearer token from the .env file

twitter_bearer_token = os.getenv("TWITTER_BEARER_TOKEN")

#Check if retrieved
type(twitter_bearer_token)

str

In [79]:
## Read the ticker symbols from CSV and put it in the dataframe
csvpath = Path("Data/StockPriceData.csv")
df_stock_price_data = pd.read_csv(csvpath, infer_datetime_format=True, parse_dates=True)

#Delete the unnecessary index column from CSV
df_stock_price_data = df_stock_price_data.drop(columns=df_stock_price_data.columns[0])
df_stock_price_data()

,timestamp,open,high,low,close,volume,trade_count,vwap,symbol
0,2022-04-11,59.06,60.335,58.6500,58.72,315665,5340,59.230175,AGO
1,2022-04-12,58.91,59.930,58.5400,58.87,289886,5709,59.070095,AGO
2,2022-04-13,58.50,60.480,58.5000,60.42,348366,6174,60.036452,AGO
3,2022-04-14,60.12,61.220,60.1200,60.40,434227,5802,60.534480,AGO
4,2022-04-18,60.12,60.970,59.8600,60.33,583988,6664,60.377456,AGO
...,...,...,...,...,...,...,...,...,...
584,2022-05-02,7.06,7.080,6.1600,6.66,242513,4136,6.601377,URGN
585,2022-05-03,6.63,6.840,6.3998,6.75,236584,4173,6.586335,URGN
586,2022-05-04,6.75,7.010,6.3400,6.91,119559,2500,6.702285,URGN
587,2022-05-05,6.84,6.900,6.2200,6.32,84668,1858,6.503618,URGN


In [78]:
#List of unique tickers
tickers_list = list(set(tickers_df['symbol']))
tickers_list

['EVOJU',
 'EPAM',
 'PZZA',
 'SECO',
 'SRL',
 'TALKW',
 'BROS',
 'FREEW',
 'MEI',
 'MTAC',
 'BEP',
 'PPYAW',
 'CTBB',
 'BLDR',
 'LMAT',
 'MOLN',
 'ALGN',
 'ERF',
 'JNCE',
 'URGN',
 'FTF',
 'SMSI',
 'SYRS',
 'GRVY',
 'HSAQ',
 'SYNL',
 'AGO',
 'GBRG',
 'TAP',
 'ATHX',
 'OGE']

In [8]:
# Set the Bearer Token for Authorisation for Twitter API call.
auth_token = "Bearer " + twitter_bearer_token
headers = {"Authorization": auth_token}

## Function to read 100 recent tweets related to ticker and from the date time specified.
def read_100_Tweets(ticker, tweet_date_time):
    ## Function to read 100 recent tweets from the specific dates
    ##  Input : ticker - Stock Ticker
    ##          tweet_date_time - UTC Date/Time Format YYYY-MM-DDTHH:mm:ssZ (ISO 8601/RFC 3339)
    ## 
    ##  Output: List of 100 tweets
    twitter_api_url  = f"https://api.twitter.com/2/tweets/search/recent?max_results=100&query={ticker}&start_time={tweet_date_time}"
    response = requests.get(twitter_api_url, headers=headers)
    ## Check for 200 status code which means it was successful
    tweets_list = [];
    if(response.status_code == 200):
        json_response = response.json()
        #Check if there are any tweets at all
        if('data' in json_response.keys()): 
            all_tweets = response.json()["data"]
            for tweet in all_tweets:
                tweets_list.append(tweet["text"])
            
    else:
        print(f"Response code: {response.status_code}.  Error in getting the tweet");
    return tweets_list    
    

In [9]:
## Perform Vader Sentiment Analysis

## Define Sentiment Object for Sentiment Analysis

sentiment_obj = SentimentIntensityAnalyzer()
def perform_sentiment_analysis(tweets_list):
    ## Function to read 100 recent tweets from the specific dates
    ##  Input : tweets_list - List of 100 tweets
    ##          
    ## 
    ##  Output: sentiment score average
    
    ##Check if there are tweets to analyse
    if (len(tweets_list) > 0):
        sentiment_scores_all = []
        for tweet in tweets_list:
            sentiment_dict = sentiment_obj.polarity_scores(tweet)
            sentiment_scores_all.append(sentiment_dict["compound"])

        #Average the sentiment of all tweets    
        average_sentiment = np.average(sentiment_scores_all)
        return average_sentiment
    else:
        return 0;
    

    

In [62]:
#perform_sentiment_analysis(tweets_list)

In [11]:
start_date = datetime(2022,5,5, 13,0,0)
end_date = datetime(2022,5,11, 13,0,0)
tweet_sentiments = []

while start_date <= end_date:
    delta = timedelta(days=1)
    for ticker in tickers_list:
        print(f"Executing Tweet Analysis for {ticker} on {start_date.isoformat()}")
        tweets_list = read_100_Tweets(ticker, start_date.isoformat() + "Z")
        sentiment_score = perform_sentiment_analysis(tweets_list)
        tweet_sentiment = {}
        tweet_sentiment["ticker"] = ticker
        tweet_sentiment["date"] = start_date
        tweet_sentiment["sentiment_score"] = sentiment_score
        tweet_sentiments.append(tweet_sentiment)
        #print("Ticker: " + tweet_sentiment["ticker"] + ", Date : " + str(tweet_sentiment["date"]) + ", Sentiment Score: " + str(tweet_sentiment["sentiment_score"]))
    start_date += delta
 

In [26]:
start_date.isoformat()

'2022-05-12T13:00:00'

In [12]:
   
tweet_sentiments

[{'ticker': 'AAIT',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.09400700000000002},
 {'ticker': 'AAL',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.1424141414141414},
 {'ticker': 'AAME',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.06330400000000001},
 {'ticker': 'AAOI',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.024663000000000004},
 {'ticker': 'AAON',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.11979099999999997},
 {'ticker': 'AAPL',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.18662800000000002},
 {'ticker': 'AAVL',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': -0.04226666666666667},
 {'ticker': 'AAWW',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.45407171717171724},
 {'ticker': 'AAXJ',
  'date': datetime.datetime(2022, 5, 5, 13, 0),
  'sentiment_score': 0.22636000000000003},
 

In [42]:
stock_tweet_sentiment_df = pd.DataFrame(tweet_sentiments)
#Date Clean up
stock_tweet_sentiment_df["date"] = pd.to_datetime(stock_tweet_sentiment_df["date"].dt.date)
#Check Date Type
stock_tweet_sentiment_df.dtypes

ticker                     object
date               datetime64[ns]
sentiment_score           float64
dtype: object

In [43]:
stock_tweet_sentiment_df

,ticker,date,sentiment_score
0,AAIT,2022-05-05,0.094007
1,AAL,2022-05-05,0.142414
2,AAME,2022-05-05,0.063304
3,AAOI,2022-05-05,0.024663
4,AAON,2022-05-05,0.119791
...,...,...,...
65,AAPL,2022-05-11,0.185938
66,AAVL,2022-05-11,-0.296000
67,AAWW,2022-05-11,0.454072
68,AAXJ,2022-05-11,0.213613


In [46]:
#Sort the tweets df, reset the index after sorting and then drop the column "index"
stock_tweet_sentiment_df = stock_tweet_sentiment_df.sort_values(by=["ticker", "date"]).reset_index().drop(columns="index")
stock_tweet_sentiment_df

,ticker,date,sentiment_score
0,AAIT,2022-05-05,0.094007
1,AAIT,2022-05-06,0.094007
2,AAIT,2022-05-07,0.094007
3,AAIT,2022-05-08,0.094007
4,AAIT,2022-05-09,0.094007
...,...,...,...
65,ABAC,2022-05-07,0.104165
66,ABAC,2022-05-08,0.104165
67,ABAC,2022-05-09,0.104165
68,ABAC,2022-05-10,0.104165


In [47]:
stock_tweet_sentiment_df.to_csv("Data/stock_tweet_sentiment.csv")